In [45]:
from IslandTime import TimeSeriesConnections, retrieve_island_info, plot_shoreline_transects
import numpy as np
import pandas as pd

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [47]:
island_info['timeseries_preprocessing']['optimal time period']['dict_timeseries']['coastline_position_transect_2_waterline']['monthly'].keys()

Index(['coastline_position_transect_2_waterline',
       'sea_surface_temperature_NOAACRW', 'sea_level_anomaly',
       '2_metre_dewpoint_temperature', 'soil_temperature_level_1',
       'total_precipitation', 'evaporation', 'sea_surface_temperature',
       'mean_sea_level_pressure', 'mean_direction_of_total_swell',
       'mean_direction_of_wind_waves', 'mean_wave_direction', 'wind_speed_10m',
       'wind_direction_10m', 'wind_direction_true_10m',
       'wave_energy_of_total_swell', 'wave_energy_of_wind_waves',
       'wave_energy_of_combined_wind_waves_and_swell'],
      dtype='object')

In [48]:
island_info = retrieve_island_info('Dhakandhoo', 'Maldives', verbose=False)

ts1 = island_info['timeseries_preprocessing']['optimal time period']['dict_timeseries']['coastline_position_transect_2_waterline']['monthly']['coastline_position_transect_2_waterline']
ts2 = island_info['timeseries_preprocessing']['optimal time period']['dict_timeseries']['coastline_position_transect_2_waterline']['monthly']['wave_energy_of_combined_wind_waves_and_swell']

ts_dict = {'ts1': ts1, 'ts2': ts2}

In [67]:
dictt = TimeSeriesConnections(ts_dict).main()


-------------------------------------------------------------------
Evaluating time series connections
-------------------------------------------------------------------

[('ts1', 'ts2')]
--- Evaluating Granger causality ---
--- Evaluating cross-correlation ---
--- Evaluating correlation ---
Pearson correlation coefficient:  -0.11425104269016166
--- Evaluating cointegration ---
 t statistic: -1.78 
 p value: 0.64 
 critical p values [1%, 5%, 10%] [-4.02083922 -3.40409789 -3.09138506]


In [81]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots()
ax2 = ax.twinx()
ts1.plot(ax=ax, color='r')
ts2.plot(ax=ax2, color='b')

<Axes: >

In [84]:
from statsmodels.tsa.stattools import ccf, grangercausalitytests, coint, adfuller

t_statistic, p_val, critical_p_val = coint(ts1, ts2, trend='ct', autolag=None, maxlag=3)
print(f' t statistic: {np.round(t_statistic, 2)} \n p value: {np.round(p_val,2)} \n critical p values [1%, 5%, 10%] {critical_p_val}')




 t statistic: -3.83 
 p value: 0.04 
 critical p values [1%, 5%, 10%] [-4.50158458 -3.88654012 -3.57500991]


In [86]:
from statsmodels.tsa.vector_ar.vecm import coint_johansen

# 4. Johansen Co-integration Test
data = pd.concat([ts1, ts2], axis=1)
result = coint_johansen(data, det_order=0, k_ar_diff=3)

print('Johansen Test Statistics:')
print(result.lr1)  # Trace statistic
print('Critical Values (90%, 95%, 99%):')
print(result.cvt)

# Interpret the Johansen test results
for i in range(len(result.lr1)):
    trace_stat = result.lr1[i]
    critical_values = result.cvt[i]
    print(f'Trace statistic for r<={i}: {trace_stat:.2f}, critical values {critical_values}')

    if trace_stat > critical_values[1]:  # Compare to 95% critical value
        print(f'Null hypothesis of r<={i} is rejected at the 95% confidence level.')
    else:
        print(f'Null hypothesis of r<={i} cannot be rejected at the 95% confidence level.')

Johansen Test Statistics:
[56.70130768 11.12405127]
Critical Values (90%, 95%, 99%):
[[13.4294 15.4943 19.9349]
 [ 2.7055  3.8415  6.6349]]
Trace statistic for r<=0: 56.70, critical values [13.4294 15.4943 19.9349]
Null hypothesis of r<=0 is rejected at the 95% confidence level.
Trace statistic for r<=1: 11.12, critical values [2.7055 3.8415 6.6349]
Null hypothesis of r<=1 is rejected at the 95% confidence level.


In [123]:
# Extract co-integrating vectors and compute the co-integrating relationship
coint_vectors = result.evec
coint_vector = coint_vectors[:, 0]  # First co-integrating vector

# data = pd.concat([ts1, ts2], axis=1)

# Calculate the co-integrating relationship
coint_relationship = np.dot(data_s, coint_vector)

# Plot the time series and co-integrating relationship on the same plot
plt.figure(figsize=(14, 6))
plt.plot(ts1, label='Time Series 1', color='blue')
plt.plot(ts2, label='Time Series 2', color='red')
plt.plot(data.index, coint_relationship, label='Co-integrating Relationship', color='green', linestyle='--')
plt.legend()
plt.title('Time Series and Co-integrating Relationship')
plt.show()

# Test for stationarity of the residuals
residuals = coint_relationship - np.mean(coint_relationship)
adf_result = adfuller(residuals)
print(f'ADF Statistic: {adf_result[0]}')
print(f'p-value: {adf_result[1]}')
print(f'Critical Values: {adf_result[4]}')

# Plot the residuals of the co-integrating relationship on a separate plot
plt.figure(figsize=(14, 6))
plt.plot(data.index, residuals, label='Residuals of Co-integrating Relationship', color='purple')
plt.axhline(0, color='black', linestyle='--')
plt.title('Residuals of Co-integrating Relationship')
plt.legend()
plt.show()

ADF Statistic: -7.971757479514952
p-value: 2.769232566258758e-12
Critical Values: {'1%': -3.5159766913976376, '5%': -2.898885703483903, '10%': -2.5866935058484217}


In [114]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.vector_ar.vecm import coint_johansen

# import plot_acf
from statsmodels.graphics.tsaplots import plot_acf

import Rbeast as rb

# Plot original time series
plt.figure(figsize=(14, 6))
plt.plot(data['coastline_position_transect_2_waterline'], label='Time Series 1')
plt.plot(data['wave_energy_of_combined_wind_waves_and_swell'], label='Time Series 2')
plt.legend()
plt.title('Original Time Series with Seasonality')
plt.show()

# Seasonal Decomposition
# decomposition_ts1 = seasonal_decompose(data['coastline_position_transect_2_waterline'], model='additive', period=12)
# decomposition_ts2 = seasonal_decompose(data['wave_energy_of_combined_wind_waves_and_swell'], model='additive', period=12)
decomposition_ts1 = rb.beast(data['coastline_position_transect_2_waterline'], period='1 year', deltat='1/12 year')
decomposition_ts2 = rb.beast(data['wave_energy_of_combined_wind_waves_and_swell'], period='1 year', deltat='1/12 year')
rb.plot(decomposition_ts2)

plot_acf(data['coastline_position_transect_2_waterline'])

# # Plot seasonal decomposition
# plt.figure(figsize=(14, 12))

# plt.subplot(2, 3, 1)
# plt.plot(data['coastline_position_transect_2_waterline'], label='Time Series 1')
# plt.legend()
# plt.title('Time Series 1')

# plt.subplot(2, 3, 2)
# plt.plot(decomposition_ts1.trend.dropna(), label='Trend Component')
# plt.legend()
# plt.title('Trend Component of TS1')

# plt.subplot(2, 3, 3)
# plt.plot(decomposition_ts1.seasonal, label='Seasonal Component')
# plt.legend()
# plt.title('Seasonal Component of TS1')

# plt.subplot(2, 3, 4)
# plt.plot(data['wave_energy_of_combined_wind_waves_and_swell'], label='Time Series 2')
# plt.legend()
# plt.title('Time Series 2')

# plt.subplot(2, 3, 5)
# plt.plot(decomposition_ts2.trend.dropna(), label='Trend Component')
# plt.legend()
# plt.title('Trend Component of TS2')

# plt.subplot(2, 3, 6)
# plt.plot(decomposition_ts2.seasonal, label='Seasonal Component')
# plt.legend()
# plt.title('Seasonal Component of TS2')

# plt.tight_layout()
# plt.show()

# # Remove seasonality by subtracting seasonal component
# data_deseasonalized = pd.DataFrame({
#     'ts1_deseasonalized': data['coastline_position_transect_2_waterline'] - decomposition_ts1.seasonal,
#     'ts2_deseasonalized': data['wave_energy_of_combined_wind_waves_and_swell'] - decomposition_ts2.seasonal
# })

# # Perform Johansen test on deseasonalized data
# result_deseasonalized = coint_johansen(data_deseasonalized, det_order=0, k_ar_diff=3)

# # Extract co-integrating vectors and compute the co-integrating relationship
# coint_vectors_deseasonalized = result_deseasonalized.evec
# coint_vector_deseasonalized = coint_vectors_deseasonalized[:, 0]  # First co-integrating vector

# # Calculate the co-integrating relationship
# coint_relationship_deseasonalized = np.dot(data_deseasonalized, coint_vector_deseasonalized)

# # Plot deseasonalized time series and co-integrating relationship
# plt.figure(figsize=(14, 6))
# # plt.plot(data_deseasonalized['ts1_deseasonalized'], label='Deseasonalized Time Series 1', color='blue')
# # plt.plot(data_deseasonalized['ts2_deseasonalized'], label='Deseasonalized Time Series 2', color='red')
# plt.plot(data.index, coint_relationship_deseasonalized, label='Co-integrating Relationship', color='green', linestyle='--')
# plt.legend()
# plt.title('Deseasonalized Time Series and Co-integrating Relationship')
# plt.show()


INFO: To supress printing the parameers in beast(),      set print.options = 0 
INFO: To supress printing the parameers in beast_irreg(),set print.options = 0 
INFO: To supress printing the parameers in beast123(),   set extra.printOptions = 0  
INFO: To supress warning messages in beast(),            set quiet = 1 
INFO: To supress warning messages in beast_irreg(),      set quiet = 1 
INFO: To supress warning messages in beast123(),         set extra.quiet = 1  

#--------------------------------------------------#
#       Brief summary of Input Data                #
#--------------------------------------------------#
Data Dimension: One signal of length 92
IsOrdered     : Yes, ordered in time
IsRegular     : Yes, evenly spaced at interval of  0.0833333 year = 1 months = 30.4167 days
HasSeasonCmpnt: True  | period = 1 year = 12 months = 365 days. The model 'Y=Trend+Season+Error' is fitted.
              : Num_of_DataPoints_per_Period = period/deltaTime = 1/0.0833333 = 12
HasOutlier

<Figure size 640x480 with 1 Axes>

In [134]:
data_s = pd.DataFrame({'ts1_s': decomposition_ts1.season.Y, 'ts2_s': decomposition_ts2.season.Y}, index=ts1.index)
data_res = pd.DataFrame({'ts1_res': ts1 - decomposition_ts1.trend.Y - decomposition_ts1.season.Y, 'ts2_res': ts2 - decomposition_ts2.trend.Y - decomposition_ts2.season.Y}, index=ts1.index)
data_t = pd.DataFrame({'ts1_t': decomposition_ts1.trend.Y, 'ts2_t': decomposition_ts2.trend.Y}, index=ts1.index)

In [143]:
fig, ax = plt.subplots()
ax.plot(ts1.index, decomposition_ts2.trend.Y)
ax2 = ax.twinx()
ax2.plot(ts1.index, decomposition_ts1.trend.Y, color='r')

In [145]:
rb.plot(decomposition_ts2)

(<Figure size 640x480 with 9 Axes>,
 array([<Axes: xlabel='[]', ylabel='Y'>,
        <Axes: xlabel='[]', ylabel='season'>,
        <Axes: xlabel='[]', ylabel='Pr(scp)'>,
        <Axes: xlabel='[]', ylabel='sOrder'>,
        <Axes: xlabel='[]', ylabel='trend'>,
        <Axes: xlabel='[]', ylabel='Pr(tcp)'>,
        <Axes: xlabel='[]', ylabel='tOrder'>,
        <Axes: xlabel='[]', ylabel='slpsgn'>,
        <Axes: xlabel='time', ylabel='error'>], dtype=object))

In [141]:
from statsmodels.tsa.vector_ar.vecm import coint_johansen

# 4. Johansen Co-integration Test
result = coint_johansen(data_t, det_order=0, k_ar_diff=1)

print('Johansen Test Statistics:')
print(result.lr1)  # Trace statistic
print('Critical Values (90%, 95%, 99%):')
print(result.cvt)


# Interpret the Johansen test results
for i in range(len(result.lr1)):
    trace_stat = result.lr1[i]
    critical_values = result.cvt[i]
    print(f'Trace statistic for r<={i}: {trace_stat:.2f}, critical values {critical_values}')

    if trace_stat > critical_values[1]:  # Compare to 95% critical value
        print(f'Null hypothesis of r<={i} is rejected at the 95% confidence level.')
    else:
        print(f'Null hypothesis of r<={i} cannot be rejected at the 95% confidence level.')


Johansen Test Statistics:
[10.22817199  2.17772393]
Critical Values (90%, 95%, 99%):
[[13.4294 15.4943 19.9349]
 [ 2.7055  3.8415  6.6349]]
Trace statistic for r<=0: 10.23, critical values [13.4294 15.4943 19.9349]
Null hypothesis of r<=0 cannot be rejected at the 95% confidence level.
Trace statistic for r<=1: 2.18, critical values [2.7055 3.8415 6.6349]
Null hypothesis of r<=1 cannot be rejected at the 95% confidence level.
